In [ ]:
"""

This script:
takes a set of nodes (external_testset.txt)
runs incremetal random walks on a chosen network (networkfile.txt)
and prints out the results in individual txts (outfile.txt)

Format requirements:
external_testset.txt: tab separated file of a header and entrezIDs
header entrezID1 entrezID2 entrezID3

networkfile.txt: an edgelist of entrezIDs
entrezID1 entrezID2
entrezID1 entrezID3
entrezID3 entrezID4


how to run this code:
python3 rndWalk_fromSet_choose_network.py 0 .9 1. external_testset.txt networkfile.txt outfile.txt

best run dowloded, on the command line.
"""
#import os
import sys
import numpy as np
import scipy.stats as st
import networkx as nx
import itertools as it
import random as rd
import os.path
from collections import defaultdict
import time
from sklearn.preprocessing import normalize



#########################################################################
#
#          CALCULATES THE RANDOM WALK OPERATOR 
#             REQUIRES ADJACENCY MATRIX
#
#########################################################################


def rnd_walk_matrix(A, r, alpha, num_nodes):
    
    n = num_nodes
    M = normalize(A, norm='l1', axis=0)                                 # column wise normalized MArkov matrix
    factor = float((1-alpha)/num_nodes)
#     print(factor)
    E = np.multiply(factor,np.ones([num_nodes,num_nodes]))              # prepare 2nd scaling term
    M2 = np.multiply(alpha,M) + E                                       # mixture of Markov chains
    del M
    del E
    
    U = np.identity(n,dtype=int) 
    H = (1-r)*M2
    H1 = np.subtract(U,H)
    del U
    del M2
    del H    

    W = r*np.linalg.inv(H1)                                             # calculate random-walk matrix
    del H1
    return W

"""
=================================================================

           E N D    O F    D E F I N I T I O N S 

=================================================================
"""



if __name__ == '__main__':
    

    
    # DECIDES WHETHER IT INVERTS MATRIX (flag = '0') OR GO FOR PRECALCULATED OPERATOR (flag = '1')
    
    # RANDOM WALK PARAMETERS - incremental R
    rlist = np.arange(.1,1,.1)
    alpha = float(sys.argv[1])
    # FILESTRING WITH GENE SET
    infile = sys.argv[2]
    # SPECIFY NETWORK 
    #(it is either an external network, provided by an edgelist of entrezIDs or
    # the PPI network - give 'ppi' as argument)
    network = sys.argv[3]
 
    fnet = open(network, 'r')
    G0 = nx.read_edgelist(network)
    # extract giant component only:
    G = max(nx.connected_component_subgraphs(G0), key=len)
    if G.number_of_nodes() < G0.number_of_nodes():
        print('Network is NOT connected! Giant component taken.')
    else:
        print('Network is connected.')
            
    fnet.close()
    print('Got alpha=%.2f and as network: %s.' %(alpha,network))
    # print('original network: #nodes=%s , # edges=%s ' %(G0.number_of_nodes(),G0.number_of_nodes()))
    print('lcc network: #nodes=%s , # edges=%s ' %(G.number_of_nodes(),G.number_of_edges()))
    
    # ###########################################################################
    #                                                                           #
    #    CALCULATES MATRIX INVERSION FOR GIVEN PARAMETERS                       #
    #                                                                           #
                                                                                #
    num_nodes = G.number_of_nodes()                                             #
    A = nx.adjacency_matrix(G, sorted(G.nodes()))                               #
                                                                                #
    #################################################################           #
    #                                                                           #
    #    INVERT MARKOV MATRIX & GENERATE RW MATRIX                              #
    #                                                                           #
    ###################                                                         #
    for r in rlist:
        t0 = time.time()
        W = rnd_walk_matrix(A, r, alpha, num_nodes)                                 #
                                                                                #
        print('Inversion done, with r== %s' %(r))                                   #
        #################################################################           #
        #                                                                           #
        #    GENERATE DICT FOR NODE-LABELS TO INTEGERS                              #
        #                                                                           #
        ###################                                                         #
                                                                                #
        d_idx_entz = {}                                                             #
        cc = 0                                                                      #
        for entz in sorted(G.nodes()):                                              #
            d_idx_entz[cc] = entz                                                   #
            cc += 1                                                                 #
        d_entz_idx = dict((y,x) for x,y in d_idx_entz.items())                      #
        print('rnd walk computing time: %.2f' %float(time.time()-t0))               #
                                                                                #
        #############################################################################




        # LOAD EXTERNAL GENE SET
        f = open(infile,'r')

        lines = f.readlines()#.split('\n')

        l_ent = []
        for line in lines:
            if '#' in line:
                continue
            ent = line.strip()
            l_ent.append(ent)
        f.close()

        if (set(l_ent).issubset(set(G.nodes()))):
            print('All genes in network.')
            l_ent2cont = l_ent
        else:
            l_ent2cont = []
            for g in l_ent:
                if g in G:
                    l_ent2cont.append(g)
                else:
                    print('Genes %s is not part of the network and will be neglected!' %g)
                
        print('number of genes in set: %s' %len(l_ent))
        print('number of set genes on network: %s' %len(l_ent2cont))

        # make list of indices
        nodeset = []
        for s_ent in l_ent2cont:
            nodeset.append(d_entz_idx[s_ent])

        normprob = len(nodeset)
        print('Size of initial set: %s seed genes' %normprob)
        p0 = np.zeros(G.number_of_nodes())
        # generate start vector
        for n in range(len(p0)):
            if n in nodeset:
                p0[n] = 1.
        print('check for sum of initial p-vec')
        print(np.sum(p0))
        pinf =  np.array(W.dot(p0))
   
        # DICT WITH NODE ID AND PVIS
        d_n_p = {}
        i = 0
        for x in pinf[0]:
            #     print(i,x)
            d_n_p[d_idx_entz[i]] = x/normprob
            i += 1
    
        # write file
        outfile = 'rwr_net_%s.1f.txt' %r
        f_out = open(outfile,'w')
        f_out.write('entrez_id\tvisit_probab_r%s\ttypegene\n' %(r))

        pcum = 0
        for node, pvis in sorted(d_n_p.items(), key = lambda x: x[1], reverse = True):
            if node in l_ent2cont:
                gtype = 'setgene'
            else:
                gtype = 'netgene'
            
            f_out.write('%s\t%s\t%s\n' %(node,pvis,gtype))
        f_out.close()


        print('total computing time: %.2f' %float(time.time()-t0))
        print('Output file written -> %s' %outfile)
